<a href="https://colab.research.google.com/github/thapawan/ImageSegmentation/blob/main/ImageSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from skimage.segmentation import flood, flood_fill
from sklearn.cluster import KMeans

def preprocess_landsat8(band3_path, band5_path):
    # Load Landsat 8 Band 3 and Band 5 images
    band3 = tiff.imread(band3_path)
    band5 = tiff.imread(band5_path)
    return band3, band5

def calculate_nwi(band3, band5):
    # Calculate Normalized Water Index (NWI)
    nwi = np.true_divide((band3 - band5), (band3 + band5))
    return nwi

def region_growing(image, seed_point, num_clusters):
    # Remove NaN values from the image
    image[np.isnan(image)] = 0

    # Perform K-means clustering on the image
    flattened_image = image.reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(flattened_image)
    labels = kmeans.labels_.reshape(image.shape)

    # Perform region growing starting from the seed point
    filled = flood_fill(image, seed_point, new_value=255, connectivity=1, in_place=False)

    # Get the mask of the region of interest based on the labeled clusters
    mask = np.zeros_like(image, dtype=np.bool)
    for label in np.unique(labels):
        if label != 0:  # Exclude the background label
            mask |= labels == label

    # Apply the mask to the filled region
    segmented_image = filled.copy()
    segmented_image[~mask] = 0

    return segmented_image

# Example usage
band3_path = '/content/L8_2022_B3.tif'  # Replace with the actual path to Landsat 8 Band 3
band5_path = '/content/L8_2022_B5.tif'  # Replace with the actual path to Landsat 8 Band 5
seed_point = (100, 100)  # Coordinates of the seed point
num_clusters = 2  # Number of clusters for K-means
threshold = 0.3  # Threshold value for binary image conversion

# Preprocess Landsat 8 Band 3 and Band 5
band3, band5 = preprocess_landsat8(band3_path, band5_path)

# Calculate Normalized Water Index (NWI)
nwi = calculate_nwi(band3, band5)

# Display the NWI image
plt.imshow(nwi, cmap='jet')
plt.colorbar()
plt.title('Normalized Water Index (NWI)')
plt.show()

# Convert NWI to a binary image using the threshold
#binary_image = np.where(nwi >= threshold, 255, 0).astype(np.uint8)

# Display the binary image
##plt.title('Binary Image')
#plt.show()


# Apply the region growing algorithm to NWI
segmented_image = region_growing(nwi, seed_point, num_clusters)
#plt.imshow(segmented_image, cmap='gray', vmin=0, vmax=1)
#plt.title('Segmented Image')
#plt.axis('off')
#plt.show()

# Generate a binary image with water as 1 and other pixels as 0
binary_image = np.zeros_like(segmented_image, dtype=np.uint8)
binary_image[segmented_image > 0] = 1
plt.imshow(binary_image)
plt.title('Binary Image')
plt.show()

# Save the binary image as TIFF
output_path = '/content/Segment_image5.tif'  # Replace with the desired output path
tiff.imwrite(output_path, binary_image)